# Text Encoding

Trong quá trình xử lý dữ liệu thực tế, rất nhiều thuộc tính (cột) xuất hiện dưới dạng văn bản như tên danh mục, trạng thái, mô tả, bình luận,… Tuy nhiên, các mô hình Machine Learning không thể làm việc trực tiếp với dữ liệu dạng text. Vì vậy, bước mã hóa dữ liệu (Text Encoding) là một phần quan trọng trong giai đoạn tiền xử lý dữ liệu (data preprocessing).

Notebook này trình bày các kỹ thuật phổ biến để chuyển đổi văn bản thành dạng số, giúp mô hình có thể hiểu và học từ dữ liệu:

1. One-Hot Encoding – biến mỗi giá trị phân loại thành một vector nhị phân
2. Target Encoding – mã hoá theo mục tiêu

# 0. Nguồn dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import unidecode

df = pd.read_csv('../data/interim/combined_interim.csv')
THRES_SHOLD = 0.005

fields = ['origin', 'status', 'brand', 'series', 'fuel_type', 'style']

# for col in fields:
#     print("\n" + "="*50)
#     print(f"THỐNG KÊ FIELD: {col.upper()}")
#     print("="*50)
#
#     vc = df[col].value_counts(dropna=False)
#
#     print(f"➡ Số giá trị unique: {df[col].nunique(dropna=False)}")
#
#     print(vc.to_string())
#     print("="*50 + "\n")


# 1. One-hot field
Một số trường dữ liệu có loại dữ liệu thuộc một tập hữu hạn trong khoảng dưới 30 loại (có thể chọn đến 50) sẽ được encoding theo one-hot

## 1.1. Mapping

Ít dữ liệu sử dụng mapping trực tiếp là được

In [ ]:
def normalize_categorical(x, mapping=None, keywords_map=None):
    """
    x: giá trị cần chuẩn hóa
    mapping: dict {giá trị chuẩn hóa: value mới}, khớp exact
    keywords_map: dict {từ khóa: value mới}, nếu từ khóa xuất hiện trong x -> map
    """
    if pd.isna(x):
        return np.nan

    val = str(x).lower().strip()

    # 1. khớp exact mapping
    if mapping and val in mapping:
        return mapping[val]

    # 2. khớp theo từ khóa
    if keywords_map:
        for kw, kw_val in keywords_map.items():
            if kw in val:
                return kw_val

    # 3. fallback
    return val

### 1.1.1 Status

In [ ]:
status_mapping = {
    'đã sử dụng': 'used',
    'xe đã dùng': 'used',
    'xe cu': 'used',
    'cũ': 'used',
    'xe mới': 'new',
    'mới': 'new'
}

df['status'] = df['status'].apply(lambda x: normalize_categorical(x, mapping=status_mapping))
print(df['status'].value_counts(dropna=False))

### 1.1.2. Fuel

In [ ]:
fuel_mapping = {
    'xăng': 'gasoline',
    'dầu': 'diesel',
    'dầu diesel': 'diesel',
    'điện': 'electric',
    'hybrid': 'hybrid'
}

df['fuel_type'] = df['fuel_type'].apply(lambda x: normalize_categorical(x, mapping=fuel_mapping))
print(df['fuel_type'].value_counts(dropna=False))

### 1.1.3. Gear

In [ ]:
gear_mapping = {
    'số sàn': 'mt',
    'manual': 'mt',
    'số tự động': 'at',
    'tự động': 'at',
    'automatic': 'at',
    'auto': 'at'
}

# từ khóa phụ, nếu muốn match chuỗi phức tạp
keywords_map = {
    'sàn': 'mt',
    'manual': 'mt',
    'tự động': 'at',
    'auto': 'at',
    'at': 'at'
}

df['gear'] = df['gear'].apply(lambda x: normalize_categorical(x, mapping=gear_mapping, keywords_map=keywords_map))
print(df['gear'].value_counts(dropna=False))

### 1.1.4. Style

In [ ]:
style_mapping = {
    'suv / cross over': 'suv',
    'suv': 'suv',
    'sedan': 'sedan',
    'hatchback': 'hatchback',
    'minivan (mpv)': 'minivan',
    'van/minivan': 'minivan',
    'van': 'minivan',
    'bán tải / pickup': 'pickup',
    'pick-up (bán tải)': 'pickup',
    'coupe': 'coupe',
    'coupe (2 cửa)': 'coupe',
    'convertible/cabriolet': 'convertible',
    'kiểu dáng khác': 'other',
}

df['style'] = df['style'].apply(lambda x: normalize_categorical(x, mapping=style_mapping))

## 1.2. Gom Nhóm

Nhiều cột hơn cần gom các mẫu rất nhỏ tránh làm sai mô hình

In [ ]:
def normalize_and_group(df, col, threshold_ratio=0.005, other_label='other'):
    """
    Chuẩn hóa tên và gom nhóm các giá trị ít xuất hiện thành 'other'.

    df: DataFrame
    col: tên cột cần xử lý
    threshold_ratio: tỷ lệ tối thiểu (ví dụ 0.005 = 0.5%)
    other_label: nhãn cho các giá trị hiếm
    """
    # 1. Chuẩn hóa tên: lowercase, không dấu, replace space bằng _
    df[col] = df[col].apply(lambda x: unidecode.unidecode(str(x)).lower().replace(' ', '_'))

    # 2. Đếm tần suất
    counts = df[col].value_counts()

    # 3. Xác định threshold
    total_samples = len(df)
    threshold = threshold_ratio * total_samples

    # 4. Gom nhóm các giá trị ít xuất hiện
    df[col] = df[col].apply(lambda x: x if counts[x] >= threshold else other_label)

    # 5. Trả về DataFrame đã xử lý
    return df

### 1.2.1. Origin

In [ ]:
df = normalize_and_group(df, 'origin', threshold_ratio=THRES_SHOLD)

# Kiểm tra kết quả
print(df['origin'].value_counts())

### 1.2.2. Location

In [ ]:
df = normalize_and_group(df, 'location', threshold_ratio=THRES_SHOLD)

# Kiểm tra kết quả
print(df['location'].value_counts())

### 1.2.3. Brand

In [ ]:
df = normalize_and_group(df, 'brand', threshold_ratio=THRES_SHOLD)

# Kiểm tra kết quả
print(df['brand'].value_counts())

# 2. Target Encoding

Đọ đa dạng của series rất lớn hơn 1300 loại không thể dùng One-hot, ta sử dụng target encoding

# 3. Lưu thay đổi

In [ ]:
# === One-Hot Encoding ===
onehot_cols = ['status', 'fuel_type', 'origin', 'style', 'brand', 'gear', 'location']

df = pd.get_dummies(df, columns=onehot_cols, prefix=onehot_cols, dtype=int)

# === Chuẩn hóa kiểu dữ liệu số ===
numeric_cols = ['price', 'odometer', 'seats', 'year']

for col in numeric_cols:
    if col not in df.columns:
        continue

    # Price luôn int64
    if col == 'price':
        df[col] = df[col].astype('int64')

    # Các cột còn lại: nếu có NaN -> dùng Int64; nếu không -> int64
    else:
        df[col] = df[col].astype('Int64') if df[col].isna().any() else df[col].astype('int64')

# === Xuất CSV ===
csv_file = '../data/interim/encoding_interim.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')

# === Thông tin ===
print("\n✓ Đã áp dụng One-Hot Encoding")
print(f"Số cột sau One-Hot Encoding: {df.shape[1]}")
print("\nCác cột one-hot tạo ra:")
print([c for c in df.columns if any(c.startswith(prefix + "_") for prefix in onehot_cols)])
print("\nDữ liệu sau encoding:")
print(df.head())